In [23]:
%matplotlib widget
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import scipy.optimize
import matplotlib.pyplot as plt
import matplotlib as mpl
from colorspacious import cspace_convert

In [24]:
mpl.rcParams["figure.dpi"] = 144
plt.style.use('dark_background')

In [9]:
jch = (20,50,0)

ciecam_color = cspace_convert(jch, "JCh", "CIECAM02")
ciecam_color

JChQMsH(J=20.00000000000001, C=50.000000000000036, h=2.528641304780442e-14, Q=55.31918440250145, M=36.14619347463442, s=80.83387532760875, H=380.21351847000267)

In [8]:
def add_alpha(im):
    im = np.asanyarray(im)
    if im.shape[-1]!=3:
        raise ValueError("Input image appears to have %d "
                         "color channels, "
                         "should have exactly 3" % im.shape[-1])
    new_shape = im.shape[:-1]+(4,)
    new_im = ma.zeros(new_shape, dtype=im.dtype)
    new_im[...,:3] = im
    new_im[...,3] = 1
    new_im[ma.any(ma.getmaskarray(im),axis=-1),3] = 0
    return new_im
def mask_image(im,limit=None):
    c = ~np.all(np.isfinite(im),axis=-1)
    c |= np.any(im<0, axis=-1)
    if limit is not None:
        c |= np.any(im>limit, axis=-1)
    mim = ma.array(im)
    mim[c,:] = ma.masked
    return mim

In [49]:
# axes = "JChQMsH"
angular = "h"

ranges = dict(
    J=(0,100),
    C=(0,100),
    h=(0,360),
    Q=(0,100),
    M=(0,100),
    s=(0,100),
    H=(0,399.9),
)
    
def color_array(c, fixed="J", x="C", y="h"):
    ac = fixed+x+y
    v = np.zeros((400,401,3))
    v[:,:,0] = getattr(c,fixed)
    c_x = getattr(c,x)
    c_y = getattr(c,y)
    v[:,:,1] = np.linspace(ranges[x][0],
                           ranges[x][1],
                           v.shape[0]
                          )[:,None]
    v[:,:,2] = np.linspace(ranges[y][0],
                           ranges[y][1],
                           v.shape[1]
                          )[None,:]
    return (add_alpha(mask_image(cspace_convert(v,ac,"sRGB1"),limit=1)),
            (ranges[x][0],ranges[x][1],ranges[y][0],ranges[y][1]),
            (c_x,c_y))
x = "C"
y = "h"
fixed = "J"
clicks = []
def color_plot(ax, ciecam_color, fixed="J", x="C", y="h"):
    srgba, extent, pt = color_array(ciecam_color,x=x,y=y,fixed=fixed)
    ax.imshow(srgba.transpose((1,0,2)), 
              extent=extent,
              origin="lower")
    ax.plot(pt[0],pt[1],"o")
    ax.set_aspect("auto")
    ax.set_xlabel(x)
    ax.set_ylabel(y)
    ax.set_title("%s fixed at %.1f" % (fixed,getattr(ciecam_color,fixed)))
    def onclick(event):
        global clicks
        if event.inaxes == ax:
            clicks.append((event.xdata, event.ydata))
        
    ax.get_figure().canvas.mpl_connect('button_press_event', onclick)
    
fig, ax = plt.subplots()
color_plot(ax, ciecam_color, fixed=fixed, x=x, y=y)


/home/archibald/software/anaconda2/envs/colors/lib/python3.7/site-packages/ipykernel_launcher.py:52: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
clicks

[]

In [51]:
fig, ax = plt.subplots()
color_plot(ax, ciecam_color, fixed="J", x="H", y="s")

/home/archibald/software/anaconda2/envs/colors/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …